In [898]:
from numpy import ones,vstack
from numpy.linalg import lstsq
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore')

# Low GP Penalty, Year 2 Year Avgs, 


In [899]:
def calculatingFormula(x, x1):
    points = [(x,MAX_RATING),(x1,MIN_RATING)]  
    x_coords, y_coords = zip(*points)
    A = vstack([x_coords,ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords)[0]
    return [m, c]

def pointSlopeValue(x, m, b):
    y = (m)*(x) + (b)
    return y

def statPerGame(columnHeader, currentCol):
    # Get per game metric
    tempArray = []
    for index, row in df.iterrows():
        tempArray.append(row[currentCol] / row.G)

    # Add per game metric to data frame
    df[columnHeader] = tempArray

    # Calculate rating for stat
    statRating(columnHeader)

def statRating(columnHeader):
    # Sort data for given stat
    sortedFor = df.sort_values(by=[columnHeader], ascending=False)

    # Calculate slope equation values
    formulaVals = calculatingFormula(sortedFor[columnHeader].iloc[0], sortedFor[columnHeader].iloc[len(df.index) - 1])

    # Get rating for each player using equation
    rating = []
    for index, row in df.iterrows():
        rating.append(int(pointSlopeValue(row[columnHeader], formulaVals[0], formulaVals[1])))

    # Add ratings to data frame
    df[((columnHeader + 'Rating'))] = rating
    
def scaledRating(columnHeader):
    # Sort data for given stat
    sortedFor = df.sort_values(by=[columnHeader], ascending=False)

    # Calculate slope equation values
    formulaVals = calculatingFormula(sortedFor[columnHeader].iloc[0], sortedFor[columnHeader].iloc[len(df.index) - 1])

    # Get rating for each player using equation
    rating = []
    for index, row in df.iterrows():
        rating.append(int(pointSlopeValue(row[columnHeader], formulaVals[0], formulaVals[1])))

    # Add ratings to data frame
    df[(("scaled" + columnHeader))] = rating

    

In [900]:
def calculateForYears(df, fileName):

    # Setting Stats and Per Games
    perGameHeaders = ["DRBPG", "DRB", "BPG", "BLK", "SPG", "STL", "PPG", "PTS", "APG", "AST", "ORBPG", "ORB"]
    headers = ["STLpc", "BLKpc", "DWS", "DBPM", "ASTpc", "OWS", "TSpc", "PER", "OBPM", "PTS", "VORP", "USGpc", "TPAr"]

    for x in range(0, len(perGameHeaders), 2):
        statPerGame(perGameHeaders[x], perGameHeaders[x + 1])

    for x in headers:
        statRating(x)

    # Get Defensive Ratings
    defensiveRating = []
    for index, row in df.iterrows():
        defensiveRate = int(((row.BPGRating * defense["blkWeight"]) + (row.SPGRating * defense["stlWeight"]) + (row.DWSRating * defense["dwsWeight"]) + (row.DBPMRating * defense["dbpmWeight"]) + (row.STLpcRating * defense["stlWeight"]) + (row.BLKpcRating * defense["blkpcWight"]) + (row.DRBPGRating * defense["drbWeight"])) / defSum)
        defensiveRating.append(defensiveRate)

    df['defensiveRating'] = defensiveRating

    # Get Offensive Ratings
    offensiveRating = []
    for index, row in df.iterrows():
        offensiveRate = int(((row.PPGRating * offense["ppgWeight"]) + (row.APGRating * offense["astWeight"]) + (row.TSpcRating * offense["tspcWeight"]) + (row.OWSRating * offense["owsWeight"]) + (row.PERRating * offense["perWeight"]) + (row.OBPMRating * offense["obpmWeight"]) + (row.ASTpcRating * offense["astpcWeight"]) + (row.PTSRating * offense["ptsWeight"]) + (row.USGpcRating * offense["usgpcWeight"])) / offSum)
        offensiveRating.append(offensiveRate)

    df['offensiveRating'] = offensiveRating

    # Calculate Player Ratings
    scaledRating("defensiveRating")
    scaledRating("offensiveRating")

    playerRating = []
    for index, row in df.iterrows():
        playerRate = int(((row.offensiveRating * 1.15) + (row.defensiveRating * .85)))
        playerRating.append(playerRate)

    df['playerRating'] = playerRating

    # Export To CSV
    sortedBy = df.sort_values(by=["playerRating"], ascending=False)

    sortedBy.to_csv("ratings/withRatings_" + fileName)

In [901]:
# Off
offense = {
    "orbWeight": 1,
    "astWeight": 1,
    "ppgWeight": 2.5,
    "astpcWeight": .5,
    "owsWeight": 2,
    "tspcWeight": .5,
    "perWeight": 3.5,
    "obpmWeight": 4.5,
    "ptsWeight": 2, 
    "usgpcWeight": .5
}


# Def
defense = {
    "drbWeight": 1,
    "stlWeight": 1,
    "blkWeight": 1,
    "stlpcWeight": .5,
    "blkpcWight": .5,
    "dwsWeight": 3,
    "dbpmWeight": 3.5,
}

offSum = 0
defSum = 0
for x in offense:
    offSum += offense[x]
print("Offensive Weighted Sum: " + str(offSum))
for x in defense:
    defSum += defense[x]
print("Defensive Weighted Sum: " + str(defSum))

MAX_RATING = 97
MIN_RATING = 68

Offensive Weighted Sum: 18.0
Defensive Weighted Sum: 10.5


In [902]:
years = list(range(2004, 2023))

for year in years:
    fileName = "stats_{}.csv".format(year)

    df = pd.read_csv("csvStats/" + fileName)
    df.dropna(inplace=True) # Drops NaN
    df.columns=df.columns.str.replace('%', 'pc') # Replace % symbols w/ pc
    df.columns=df.columns.str.replace('3', 'T') # Replace 3 with T

    # Drop players who played less than 20 games
    for index, row in df.iterrows():
        if row.G < 20:
            print(row.Player + " dropped")
            df.drop(index, inplace=True)

    # Calculate Rating For Each Year
    calculateForYears(df, fileName)

Troy Bell dropped
Curtis Borchardt dropped
Jamison Brewer dropped
Matt Carroll dropped
Maurice Carter dropped
Mateen Cleaves dropped
Hubert Davis dropped
Josh Davis dropped
Derrick Dial dropped
Bryce Drew dropped
Desmond Ferguson dropped
Anthony Goldwire dropped
Adrian Griffin dropped
Shane Heal dropped
James Jones dropped
Raef LaFrentz dropped
Dan Langhi dropped
Rusty LaRue dropped
Randy Livingston dropped
Darrick Martin dropped
Jamal Mashburn dropped
Tracy Murray dropped
Glen Rice dropped
Paul Shirley dropped
Jeff Trepagnier dropped
Ime Udoka dropped
Chris Whitney dropped
Wang Zhizhi dropped
Cory Alexander dropped
Jonathan Bender dropped
Michael Bradley dropped
Jamison Brewer dropped
Damone Brown dropped
Kedrick Brown dropped
Mateen Cleaves dropped
Derrick Coleman dropped
Omar Cook dropped
Kaniel Dickens dropped
Ndudi Ebi dropped
Luis Flores dropped
Tremaine Fowlkes dropped
Alex Garcia dropped
Devean George dropped
Kendall Gill dropped
Dion Glover dropped
Marcus Haislip dropped
Luke 

In [903]:
# perGameHeaders = ["DRBPG", "DRB", "BPG", "BLK", "SPG", "STL", "PPG", "PTS", "APG", "AST", "ORBPG", "ORB"]
# headers = ["STLpc", "BLKpc", "DWS", "DBPM", "ASTpc", "OWS", "TSpc", "PER", "OBPM", "PTS", "VORP", "USGpc", "TPAr"]

# for x in range(0, len(perGameHeaders), 2):
#     statPerGame(perGameHeaders[x], perGameHeaders[x + 1])

# for x in headers:
#     statRating(x)


In [904]:
# defensiveRating = []
# for index, row in df.iterrows():
#     defensiveRate = int(((row.BPGRating * defense["blkWeight"]) + (row.SPGRating * defense["stlWeight"]) + (row.DWSRating * defense["dwsWeight"]) + (row.DBPMRating * defense["dbpmWeight"]) + (row.STLpcRating * defense["stlWeight"]) + (row.BLKpcRating * defense["blkpcWight"]) + (row.DRBPGRating * defense["drbWeight"])) / defSum)
#     defensiveRating.append(defensiveRate)

# df['defensiveRating'] = defensiveRating


In [905]:
# offensiveRating = []
# for index, row in df.iterrows():
#     offensiveRate = int(((row.PPGRating * offense["ppgWeight"]) + (row.APGRating * offense["astWeight"]) + (row.TSpcRating * offense["tspcWeight"]) + (row.OWSRating * offense["owsWeight"]) + (row.PERRating * offense["perWeight"]) + (row.OBPMRating * offense["obpmWeight"]) + (row.ASTpcRating * offense["astpcWeight"]) + (row.PTSRating * offense["ptsWeight"])) / offSum)
#     offensiveRating.append(offensiveRate)

# df['offensiveRating'] = offensiveRating


In [906]:
# scaledRating("defensiveRating")
# scaledRating("offensiveRating")

# playerRating = []
# for index, row in df.iterrows():
#     playerRate = int(((row.offensiveRating) + (row.defensiveRating)))
#     playerRating.append(playerRate)

# df['playerRating'] = playerRating

# # scaledRating("playerRating")

# # sortedBy = df.sort_values(by=['scaledplayerRating'], ascending=False)

In [907]:
# sortedBy = df.sort_values(by=["playerRating"], ascending=False)

# sortedBy.to_csv("ratings/withRatings_" + fileName)

# for index, row in sortedBy.iterrows():
#     string = "{:>24} | Rating: {} | Off Rating: {} | Def Rating: {} |  Games Played: {}"
#     print(string.format(row.Player,str(row.playerRating),str(row.offensiveRating),str(row.defensiveRating),str(row.G)))

In [908]:
finalDataFrame = pd.DataFrame()

for year in years:
    fileName = "ratings/withRatings_stats_{}.csv".format(year)
    df = pd.read_csv(fileName)

    data = []

    for index, row in df.iterrows():
        data.append([row.Player, year, row.Pos, row.Age, row.Tm, row.scaledoffensiveRating, row.scaleddefensiveRating, row.playerRating])

    tempDataFrame = pd.DataFrame(data, columns=['PlayerName','Season', 'Position', 'Age', 'Team', 'OffensiveRating', 'DefensiveRating', 'RatingSum'])

    finalDataFrame = pd.concat([finalDataFrame, tempDataFrame])

df = finalDataFrame

scaledRating("RatingSum")

sortedBy = finalDataFrame.sort_values(by=["scaledRatingSum"], ascending=False)

sortedBy.to_csv("ratingsOverTime.csv")

In [912]:
for year in years:
    dataFrame = df.loc[df['Season'] == year]

    dataFrame.to_csv("finalRatings/fr_" + str(year) + ".csv")
